In [1]:
import json
import pandas as pd

# Salto

In [2]:
with open('../data/data_salto/concursos/concursos_salto_internacional_2025.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 149
La longitud de Categoría es: 149
La longitud de Provincia es: 149
La longitud de Localidad es: 149
La longitud de Disciplina es: 149
La longitud de Federación es: 149
La longitud de Resultados es: 149
La longitud de País es: 149
La longitud de Inicio es: 148
La longitud de Final es: 148
La longitud de Ambito es: 148


In [3]:
import json

# Cargar el JSON
with open('../data/data_salto/concursos/concursos_salto_internacional_2025.json', 'r', encoding='utf-8') as file:
    datos_concursos = json.load(file)

# Obtener la longitud de referencia
longitud_maxima = max(len(v) for v in datos_concursos.values() if isinstance(v, list))

# Listas a revisar
listas_objetivo = ["Inicio", "Final", "Ambito"]

# Detectar y mostrar valores faltantes
for key in listas_objetivo:
    lista = datos_concursos[key]
    indices_presentes = set(range(len(lista)))
    indices_totales = set(range(longitud_maxima))
    indices_faltantes = sorted(indices_totales - indices_presentes)

    print(f"\n🔎 Valores faltantes en la lista '{key}':")

    for idx in indices_faltantes:
        nombre = datos_concursos["Nombre"][idx] if idx < len(datos_concursos["Nombre"]) else "(sin nombre)"
        fecha_inicio = datos_concursos["Inicio"][idx] if idx < len(datos_concursos["Inicio"]) else "(sin fecha)"
        fecha_final = datos_concursos["Final"][idx] if idx < len(datos_concursos["Final"]) else "(sin fecha)"
        print(f"- Índice {idx} → concurso: {nombre} | Inicio: {fecha_inicio} | Final: {fecha_final}")




🔎 Valores faltantes en la lista 'Inicio':
- Índice 148 → concurso: CSI1* Opglabbeek | Inicio: (sin fecha) | Final: (sin fecha)

🔎 Valores faltantes en la lista 'Final':
- Índice 148 → concurso: CSI1* Opglabbeek | Inicio: (sin fecha) | Final: (sin fecha)

🔎 Valores faltantes en la lista 'Ambito':
- Índice 148 → concurso: CSI1* Opglabbeek | Inicio: (sin fecha) | Final: (sin fecha)


In [4]:
with open('../data/data_salto/pruebas/pruebas_salto_internacional_2025.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 1144
La longitud de Fecha es: 1144
La longitud de Prueba es: 1144
La longitud de Categoría es: 1142
La longitud de Número es: 1141
La longitud de Concurso es: 1144


In [7]:
import json

# Cargar datos de pruebas
with open('../data/data_salto/pruebas/pruebas_salto_internacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Longitud total según el campo más largo
longitud_total = len(datos_pruebas["Concurso"])

# Campos a verificar
campos_objetivo = ["Prueba", "Categoría", "Fecha"]  # puedes añadir más si quieres

# Verificar faltantes y mostrar con fecha si existe
for campo in campos_objetivo:
    if campo not in datos_pruebas:
        continue
    
    lista = datos_pruebas[campo]
    indices_faltantes = set(range(longitud_total)) - set(range(len(lista)))
    
    if indices_faltantes:
        print(f"\n🔍 Faltantes en '{campo}':")
        for i in sorted(indices_faltantes):
            fecha = datos_pruebas["Fecha"][i] if i < len(datos_pruebas["Fecha"]) else "(sin fecha)"
            concurso = datos_pruebas["Concurso"][i] if i < len(datos_pruebas["Concurso"]) else "(sin concurso)"
            print(f"- Índice {i} → concurso: '{concurso}' | fecha: {fecha}")





🔍 Faltantes en 'Prueba':
- Índice 14957 → concurso: 'CSI2* LIER (BEL)' | fecha: 115-125 - Two phases, both atc T A
- Índice 14958 → concurso: 'CSI2* LIER (BEL)' | fecha: CSI YH*
- Índice 14959 → concurso: 'CSI2* LIER (BEL)' | fecha: Disponibles
- Índice 14960 → concurso: 'CSI2* LIER (BEL)' | fecha: Salto de Obstáculos
- Índice 14961 → concurso: 'CSI2* LIER (BEL)' | fecha: 16/12/2020
- Índice 14962 → concurso: 'CSI2* LIER (BEL)' | fecha: 16/12/2020
- Índice 14963 → concurso: 'CSI2* LIER (BEL)' | fecha: 17/12/2020
- Índice 14964 → concurso: 'CSI2* LIER (BEL)' | fecha: 17/12/2020
- Índice 14965 → concurso: 'CSIYH LIER (BEL)' | fecha: 18/12/2020
- Índice 14966 → concurso: 'CSIYH LIER (BEL)' | fecha: 18/12/2020
- Índice 14967 → concurso: 'CSIYH LIER (BEL)' | fecha: 19/12/2020
- Índice 14968 → concurso: 'CSIYH LIER (BEL)' | fecha: 19/12/2020
- Índice 14969 → concurso: 'CSIYH LIER (BEL)' | fecha: 20/12/2020
- Índice 14970 → concurso: 'CSI2* LIER (BEL)' | fecha: (sin fecha)
- Índice 14971 → c

In [9]:
with open('../data/data_salto/concursos/concursos_salto_internacional_2025.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 149
La longitud de Categoría es: 149
La longitud de Provincia es: 149
La longitud de Localidad es: 149
La longitud de Disciplina es: 149
La longitud de Federación es: 149
La longitud de Resultados es: 149
La longitud de País es: 149
La longitud de Inicio es: 148
La longitud de Final es: 148
La longitud de Ambito es: 148


In [10]:
with open('../data/data_salto/pruebas/pruebas_salto_internacional_2025.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 1144
La longitud de Fecha es: 1144
La longitud de Prueba es: 1144
La longitud de Categoría es: 1142
La longitud de Número es: 1141
La longitud de Concurso es: 1144


# Completo

In [38]:
with open('../data/data_completo/pruebas/pruebas_completo_nacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 561
La longitud de Fecha es: 561
La longitud de Prueba es: 561
La longitud de Categoría es: 561
La longitud de Número es: 561
La longitud de Concurso es: 561


In [39]:
with open('../data/data_completo/pruebas/pruebas_completo_internacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 496
La longitud de Fecha es: 496
La longitud de Prueba es: 496
La longitud de Categoría es: 496
La longitud de Número es: 496
La longitud de Concurso es: 496


In [40]:
with open('../data/data_completo/concursos/concursos_completo_internacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 480
La longitud de Categoría es: 480
La longitud de Provincia es: 480
La longitud de Localidad es: 480
La longitud de Disciplina es: 480
La longitud de Federación es: 480
La longitud de Resultados es: 480
La longitud de País es: 480
La longitud de Inicio es: 477
La longitud de Final es: 477
La longitud de Ambito es: 477


In [41]:
with open('../data/data_completo/concursos/concursos_completo_nacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 223
La longitud de Categoría es: 223
La longitud de Provincia es: 223
La longitud de Localidad es: 223
La longitud de Disciplina es: 223
La longitud de Federación es: 223
La longitud de Resultados es: 223
La longitud de País es: 223
La longitud de Inicio es: 223
La longitud de Final es: 223
La longitud de Ambito es: 223
